In [103]:
import math
import random
import numpy
from functools import reduce
import sys
import getopt
import re
import time

In [104]:
optimalFile = 'E-n101-k14.sol'
testFile = 'E-n101-k14.vrp'

In [105]:
def getDataFromFile(fileName):
    f = open(fileName, "r", encoding="utf-8")
    content = f.read()
    #print("Content: ", content)
    optimalValue = re.search("Optimal value: (\d+)", content, re.MULTILINE)
    if(optimalValue != None):
        optimalValue = optimalValue.group(1)
    else:
        optimalValue = re.search("Best value: (\d+)", content, re.MULTILINE)
        if(optimalValue != None):
            optimalValue = optimalValue.group(1)
    capacity = re.search("^CAPACITY : (\d+)$", content, re.MULTILINE).group(1)
    #graph = re.findall(r"^ (\d+) (\d+) (\d+)$", content, re.MULTILINE)
    graph = re.findall(r"^(\d+) (\d+) (\d+)$", content, re.MULTILINE)
    dim = re.search("^DIMENSION : (\d+)$", content, re.MULTILINE).group(1)
    print("Graph: ", graph)
    dim = int(dim)
    print("dim: ", dim)
    #demand = re.findall(r"^(\d+) (\d+) $", content, re.MULTILINE)
    demand = re.findall(r"^(\d+) (\d+)$", content, re.MULTILINE)
    print("Demand: ", demand)
    graph = {int(a):(int(b),int(c)) for a,b,c in graph}
    print("graph: ", graph)
    demand = {int(a):int(b) for a,b in demand}
    print("demand: ", demand)
    capacity = int(capacity)
    optimalValue = int(optimalValue)
    return capacity, graph, demand, optimalValue, dim

In [106]:
alfa = 2 # importance of pheromones
beta = 7 # importance of distance
sigm = 3 # amount of feromone
ro = 0.1 # speed of pheromon evaporation
th = 80 # initial pheromone level
iterations = 1000
ants = 14

In [107]:
def generateGraph(capacityLimit, graph, demand, optimalValue):
    vertices = list(graph.keys())
    vertices.remove(1)

    edges = { (min(a,b),max(a,b)) : numpy.sqrt((graph[a][0]-graph[b][0])**2 + (graph[a][1]-graph[b][1])**2) for a in graph.keys() for b in graph.keys()}
    feromones = { (min(a,b),max(a,b)) : 1 for a in graph.keys() for b in graph.keys() if a!=b }
    
    return vertices, edges, capacityLimit, demand, feromones, optimalValue

In [108]:
import math

In [109]:
def solutionOfOneAnt(vertices, edges, capacityLimit, demand, feromones):
    solution = list()

    while(len(vertices)!=0):
        path = list()
        city = numpy.random.choice(vertices)
        capacity = capacityLimit - demand[city]
        path.append(city)
        vertices.remove(city)
        while(len(vertices)!=0):
            probabilities = list(map(lambda x: ((feromones[(min(x,city), max(x,city))])**alfa)*((1/edges[(min(x,city), max(x,city))])**beta), vertices))
            #print(probabilities)
            for i in range(len(probabilities)):
              if math.isinf(probabilities[i]):
                probabilities[i] = 1.2224513299473679e-11
            probabilities = probabilities/numpy.sum(probabilities)
            
            city = numpy.random.choice(vertices, p=probabilities)
            capacity = capacity - demand[city]

            if(capacity>0):
                path.append(city)
                vertices.remove(city)
            else:
                break
        solution.append(path)
    return solution

def rateSolution(solution, edges):
    s = 0
    for i in solution:
        a = 1
        for j in i:
            b = j
            s = s + edges[(min(a,b), max(a,b))]
            a = b
        b = 1
        s = s + edges[(min(a,b), max(a,b))]
    return s


In [110]:
def updateFeromone(feromones, solutions, bestSolution):
    # L - cost of the destance
    Lavg = reduce(lambda x,y: x+y, (i[1] for i in solutions))/len(solutions)
    #amount of feromones
    feromones = { k : (ro + th/Lavg)*v for (k,v) in feromones.items() }
    solutions.sort(key = lambda x: x[1])
    if(bestSolution!=None):
        if(solutions[0][1] < bestSolution[1]):
            bestSolution = solutions[0]
        for path in bestSolution[0]:
            for i in range(len(path)-1):
                feromones[(min(path[i],path[i+1]), max(path[i],path[i+1]))] = sigm/bestSolution[1] + feromones[(min(path[i],path[i+1]), max(path[i],path[i+1]))]
    else:
        bestSolution = solutions[0]
    for l in range(sigm):
        paths = solutions[l][0]
        L = solutions[l][1]
        for path in paths:
            for i in range(len(path)-1):
                feromones[(min(path[i],path[i+1]), max(path[i],path[i+1]))] = (sigm-(l+1)/L**(l+1)) + feromones[(min(path[i],path[i+1]), max(path[i],path[i+1]))]
    return bestSolution

In [111]:
bestSolution = None
capacityLimit, graph, demand, optimalValue, dim = getDataFromFile(testFile)

Graph:  [('1', '35', '35'), ('2', '41', '49'), ('3', '35', '17'), ('4', '55', '45'), ('5', '55', '20'), ('6', '15', '30'), ('7', '25', '30'), ('8', '20', '50'), ('9', '10', '43'), ('10', '55', '60'), ('11', '30', '60'), ('12', '20', '65'), ('13', '50', '35'), ('14', '30', '25'), ('15', '15', '10'), ('16', '30', '5'), ('17', '10', '20'), ('18', '5', '30'), ('19', '20', '40'), ('20', '15', '60'), ('21', '45', '65'), ('22', '45', '20'), ('23', '45', '10'), ('24', '55', '5'), ('25', '65', '35'), ('26', '65', '20'), ('27', '45', '30'), ('28', '35', '40'), ('29', '41', '37'), ('30', '64', '42'), ('31', '40', '60'), ('32', '31', '52'), ('33', '35', '69'), ('34', '53', '52'), ('35', '65', '55'), ('36', '63', '65'), ('37', '2', '60'), ('38', '20', '20'), ('39', '5', '5'), ('40', '60', '12'), ('41', '40', '25'), ('42', '42', '7'), ('43', '24', '12'), ('44', '23', '3'), ('45', '11', '14'), ('46', '6', '38'), ('47', '2', '48'), ('48', '8', '56'), ('49', '13', '52'), ('50', '6', '68'), ('51', '47',

In [112]:
vertices, edges, capacityLimit, demand, feromones, optimalValue = generateGraph(capacityLimit, graph, demand, optimalValue)

In [ ]:
start_time = time.time()
for i in range(iterations):
  solutions = list()
  for _ in range(ants):
      solution = solutionOfOneAnt(vertices.copy(), edges, capacityLimit, demand, feromones)
      solutions.append((solution, rateSolution(solution, edges)))
  bestSolution = updateFeromone(feromones, solutions, bestSolution)
print("Solution: ", bestSolution) 
print("time of execution: %s seconds" %abs (time.time() - start_time)) # вычисление времени выполнения


In [ ]:
cost_from_alg = bestSolution[-1]
cost_from_alg

In [ ]:
def getResFromFile(fileName):
  f = open(fileName, "r", encoding="utf-8")
  content = f.read()
  cost = re.search("^Cost (\d+)$", content, re.MULTILINE).group(1)
  return int(cost)

In [ ]:
cost = getResFromFile(optimalFile)
cost

In [ ]:
def compareResults(res1, res2):
  return abs(res1-res2)

In [ ]:
compareResults(cost, cost_from_alg)

In [ ]:
print("Time ", (abs (time.time() - start_time)))
print("Optimal ", cost)
print("In fact ", cost_from_alg)
print("Dim ", dim)